# Hybrid Wavelets/MP

We are training a seperate model for each timestep in the lookahead window

We are denoising the input for each training example using wavelet threshold denoising, then computing the matrix profile

Due to a coding slip, the matrix profile is flipped; this is adressed in the Hybrid_Mp_Wavelets-Fwd notebook

## Load data and functions

In [20]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras import callbacks
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

import datetime 
from datetime import timedelta
from datetime import datetime
import pywt
import matrixprofile as mp

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [4]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [5]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [6]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [7]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [8]:
def generate_training_val_examples_univariate_output(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM, filter output to only the first input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))[:,:,:1]
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))[:,:,:1]

    
    return X_train, y_train, X_val, y_val, scaler


In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [10]:
def descale_y_retrun_metrics(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for each output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    for j in range(0, n_steps_out):
        RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
        for i in range(0, len(inputs)):  
            pred_descaled= (scaler.inverse_transform(yhat[:,j:j+1,:].reshape(yhat.shape[0], yhat.shape[2])))[:, i:i+1]
            groud_truth_descaled= ((scaler.inverse_transform(y_val2[:,j:j+1,:].reshape(y_val2.shape[0], y_val2.shape[2]))))[:, i:i+1]
            RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
            MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
            MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
            MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
            R2 = r2_score(groud_truth_descaled, pred_descaled)
            RMSE_list.append(RMSE)
            MAE_list.append(MAE)
            MAPE_list.append(MAPE)
            R2_list.append(R2)
            MSE_list.append(MSE)
        metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)
        dict_dfs.append(metrics_df)
        dict_indexes.append('Lookahead' +str(j))
    metrics_dict = dict(zip(dict_indexes, dict_dfs))
    return metrics_dict

In [11]:
def descale_y_retrun_metrics_univariate_y(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for single output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    for j in range(0, n_steps_out):
        pred_descaled= scaler.inverse_transform(yhat)
        groud_truth_descaled= scaler.inverse_transform(y_val2[:,0,])
        RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
        MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
        MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
        MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
        R2 = r2_score(groud_truth_descaled, pred_descaled)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)
    metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1, (n_steps_out+1)))

 
    return metrics_df

In [26]:
def LSTM_model():
    checkpoint_filepath='./Hybrid Univarite2/'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    model = Sequential()
    model.add(LSTM(units =15,activation='tanh',input_shape=((n_steps_in-288), len(inputs)+1),dropout=0.01))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    return model, model_checkpoint_callback, checkpoint_filepath

In [13]:
def add_mp_reversed(data, window):
    #Given 3d array, add matrix profile of (x,y,0) as new dimension; new array has dimensiosn (x,y,z+1) 
    mp_list=[]
    for i in data[:,:,0]:
        profile = mp.compute(np.flip(i, axis=0), window, n_jobs=4)['mp']
        #we are padding the end of the sequence with the mean
        #matrix profile is always 1 full window size smalelr than input data
        mp_list.append(np.append(profile,([mean(profile)]*(data.shape[1]-len(profile)))))
        
    #concatenate matrix profile data with original    
    mp_array = np.array(mp_list).reshape(data.shape[0], data.shape[1])
    std_array = ((mp_array-mean(mp_array))/np.std(mp_array)).reshape(data.shape[0], data.shape[1],1)
    data = np.concatenate((data, std_array), axis=2)[:, window:, :]
    
    return data

In [14]:
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [15]:
def wavelet_denoising(x, wavelet='db4', level=2, tmod=1):
    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/tmod) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(3 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

In [16]:
def denoise_array(data, variable, wavelet, level, tmod):
    denoised_examples=[]
    for i in (data[:,:,variable]):
        
        denoised_examples.append(wavelet_denoising(i, wavelet=wavelet, level=level, tmod=tmod))
        denoised_array= np.array(denoised_examples)
        denoised_array= denoised_array.reshape(denoised_array.shape[0], denoised_array.shape[1], 1)
    return np.concatenate((denoised_array, data[:,:,1:]),axis=2)

## Training loop; Hybrid + MP(reversed)

In [27]:
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#Create Training Examples for all lookaheads
resample_rate = '5T'
start_date='2021-11-26 00:00:00'
end_date='2021-12-26 23:55:00' 
inputs = ['min_gas_price']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 2016+288
n_steps_out = 1
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1

y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
mp_window=288
for month in [0, 1, 2, 3, 4]:
    n_step_lookahead=1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    for i in range(1,11):
        n_step_lookahead = i
        
        #X_train_denoised= denoise_array(X_train, 0, 'db4', 2, 3)
        #X_val_denoised= denoise_array(X_val, 0, 'db4', 2, 3)
        X_train_mp = add_mp_reversed(X_train, mp_window)
        X_val_mp = add_mp_reversed(X_val, mp_window)
     


        model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
        train_history = model.fit(X_train_mp, y_train,validation_data=(X_val_mp, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
        model.load_weights(checkpoint_filepath)
        model.save(('Hybrid MP/month' +str(month)+'_'+ str(i) + '_step_lookahead_Hybrid'))

        yhat_train = model.predict(X_train_mp, verbose=1)
        yhat_val = model.predict(X_val_mp, verbose=1)
        y_hat_list.append(yhat_val)

        train_loss_list.append(train_history.history['loss'])
        val_loss_list.append(train_history.history['val_loss'])
        pd.DataFrame(train_loss_list).to_csv('Hybrid MP_fwd/train_loss')
        pd.DataFrame(val_loss_list).to_csv('Hybrid MP_fwd/val_loss')
        
        training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
        valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))

    
        x_train_extended = []
        for j in range(0, len(X_train)):
            X_train_t = np.append(X_train[j], yhat_train[j])
            x_train_extended.append(X_train_t)

        x_val_extended = []
        for k in range(0, len(X_val)):
            X_val_t = np.append(X_val[k], yhat_val[k])
            x_val_extended.append(X_val_t)

        n_step_lookahead = i+1
        X_traindummy, y_train, X_valdummy, y_val, scaler = generate_training_val_examples_univariate_output(data)
        x_train = np.array(x_train_extended)
        x_val = np.array(x_val_extended)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
        x_val = x_val.reshape(x_val.shape[0], x_val.shape[1],1)
        x_train = x_train[:,1:]
        x_val = x_val[:,1:]
        X_train = x_train[:(y_train.shape[0]),:,:]
        X_val = x_val[:(y_val.shape[0]),:,:]
        np.save("Hybrid MP_fwd/train_metric_dict.npy", training_metrics_dicts)  
        np.save("Hybrid MP_fwd/val_metric_dict.npy", valdiation_metrics_dicts)  

        
        


D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
145/145 [==============================] - 15s 62ms/step - loss: 0.3644 - val_loss: 0.0423
Epoch 2/15
145/145 [==============================] - 8s 57ms/step - loss: 0.3063 - val_loss: 0.0385
Epoch 3/15
145/145 [==============================] - 8s 58ms/step - loss: 0.2703 - val_loss: 0.0359
Epoch 4/15
145/145 [==============================] - 8s 58ms/step - loss: 0.2474 - val_loss: 0.0333
Epoch 5/15
145/145 [==============================] - 8s 58ms/step - loss: 0.2301 - val_loss: 0.0321
Epoch 6/15
145/145 [==============================] - 8s 58ms/step - loss: 0.2175 - val_loss: 0.0301
Epoch 7/15
145/145 [==============================] - 8s 58ms/step - loss: 0.2080 - val_loss: 0.0290
Epoch 8/15
145/145 [==============================] - 9s 59ms/step - loss: 0.2007 - val_loss: 0.0294
Epoch 9/15
145/145 [==============================] - 8s 58ms/step - loss: 0.1973 - val_loss: 0.0281
Epoch 10/15
145/145 [==============================] - 8s 58ms/step - loss: 0.1922 - val_l

INFO:tensorflow:Assets written to: Hybrid MP/month0_1_step_lookahead_Hybrid\assets


INFO:tensorflow:Assets written to: Hybrid MP/month0_1_step_lookahead_Hybrid\assets


63/63 [==============================] - 1s 21ms/step


...

Epoch 1/15
145/145 [==============================] - 11s 56ms/step - loss: 1.6250 - val_loss: 0.1244
Epoch 2/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6143 - val_loss: 0.1253
Epoch 3/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6122 - val_loss: 0.1298
Epoch 4/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6119 - val_loss: 0.1300
Epoch 5/15
145/145 [==============================] - 7s 52ms/step - loss: 1.6110 - val_loss: 0.1364
Epoch 6/15
145/145 [==============================] - 7s 52ms/step - loss: 1.6093 - val_loss: 0.1508
Epoch 7/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6015 - val_loss: 0.1622
Epoch 8/15
145/145 [==============================] - 8s 52ms/step - loss: 1.5919 - val_loss: 0.1468
Epoch 9/15
145/145 [==============================] - 8s 52ms/step - loss: 1.5937 - val_loss: 0.1766
Epoch 10/15
145/145 [==============================] - 8s 52ms/step - loss: 1.5881 - val_l

INFO:tensorflow:Assets written to: Hybrid MP/month4_10_step_lookahead_Hybrid\assets


INFO:tensorflow:Assets written to: Hybrid MP/month4_10_step_lookahead_Hybrid\assets


63/63 [==============================] - 1s 19ms/step


D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [44]:
pd.DataFrame(np.load('Hybrid MP/val_metric_dict.npy', allow_pickle=True).reshape(50,5)).mean()

0     27.334770
1    932.324675
2     13.919162
3      0.184442
4      0.509187
dtype: float64

In [48]:
pd.DataFrame(np.load('Hybrid Univarite3/val_metric_dict_1.npy', allow_pickle=True).reshape(50,5)).mean()

0     26.082692
1    852.171591
2     13.094625
3      0.171413
4      0.542148
dtype: float64

## Training loop:hybrid + MP(reversed) + DB4 denoising

In [28]:
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#Create Training Examples for all lookaheads
resample_rate = '5T'
start_date='2021-11-26 00:00:00'
end_date='2021-12-26 23:55:00' 
inputs = ['min_gas_price']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 2016+288
n_steps_out = 1
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1

y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
mp_window=288
for month in [0, 1, 2, 3, 4]:
    n_step_lookahead=1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    for i in range(1,11):
        n_step_lookahead = i
        
        X_train_denoised= denoise_array(X_train, 0, 'db4', 2, 3)
        X_val_denoised= denoise_array(X_val, 0, 'db4', 2, 3)
        X_train_mp = add_mp_reversed(X_train, mp_window)
        X_val_mp = add_mp_reversed(X_val, mp_window)
     


        model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
        train_history = model.fit(X_train_mp, y_train,validation_data=(X_val_mp, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
        model.load_weights(checkpoint_filepath)
        model.save(('Hybrid MP_wav/month' +str(month)+'_'+ str(i) + '_step_lookahead_Hybrid'))

        yhat_train = model.predict(X_train_mp, verbose=1)
        yhat_val = model.predict(X_val_mp, verbose=1)
        y_hat_list.append(yhat_val)

        train_loss_list.append(train_history.history['loss'])
        val_loss_list.append(train_history.history['val_loss'])
        pd.DataFrame(train_loss_list).to_csv('Hybrid MP_wav/train_loss')
        pd.DataFrame(val_loss_list).to_csv('Hybrid MP_wav/val_loss')
        
        training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
        valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))

    
        x_train_extended = []
        for j in range(0, len(X_train)):
            X_train_t = np.append(X_train[j], yhat_train[j])
            x_train_extended.append(X_train_t)

        x_val_extended = []
        for k in range(0, len(X_val)):
            X_val_t = np.append(X_val[k], yhat_val[k])
            x_val_extended.append(X_val_t)

        n_step_lookahead = i+1
        X_traindummy, y_train, X_valdummy, y_val, scaler = generate_training_val_examples_univariate_output(data)
        x_train = np.array(x_train_extended)
        x_val = np.array(x_val_extended)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
        x_val = x_val.reshape(x_val.shape[0], x_val.shape[1],1)
        x_train = x_train[:,1:]
        x_val = x_val[:,1:]
        X_train = x_train[:(y_train.shape[0]),:,:]
        X_val = x_val[:(y_val.shape[0]),:,:]
        np.save("Hybrid MP_wav/train_metric_dict.npy", training_metrics_dicts)  
        np.save("Hybrid MP_wav/val_metric_dict.npy", valdiation_metrics_dicts)  

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
145/145 [==============================] - 10s 56ms/step - loss: 0.3582 - val_loss: 0.0410
Epoch 2/15
145/145 [==============================] - 8s 53ms/step - loss: 0.2942 - val_loss: 0.0366
Epoch 3/15
145/145 [==============================] - 8s 52ms/step - loss: 0.2624 - val_loss: 0.0320
Epoch 4/15
145/145 [==============================] - 8s 53ms/step - loss: 0.2422 - val_loss: 0.0304
Epoch 5/15
145/145 [==============================] - 8s 52ms/step - loss: 0.2277 - val_loss: 0.0357
Epoch 6/15
145/145 [==============================] - 8s 52ms/step - loss: 0.2183 - val_loss: 0.0291
Epoch 7/15
145/145 [==============================] - 8s 52ms/step - loss: 0.2111 - val_loss: 0.0399
Epoch 8/15
145/145 [==============================] - 8s 52ms/step - loss: 0.2051 - val_loss: 0.0277
Epoch 9/15
145/145 [==============================] - 8s 52ms/step - loss: 0.2039 - val_loss: 0.0278
Epoch 10/15
145/145 [==============================] - 8s 55ms/step - loss: 0.1967 - val_l

INFO:tensorflow:Assets written to: Hybrid MP_wav/month0_1_step_lookahead_Hybrid\assets


INFO:tensorflow:Assets written to: Hybrid MP_wav/month0_1_step_lookahead_Hybrid\assets


63/63 [==============================] - 1s 19ms/step


...

Epoch 1/15
145/145 [==============================] - 11s 57ms/step - loss: 1.6425 - val_loss: 0.1336
Epoch 2/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6194 - val_loss: 0.1345
Epoch 3/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6166 - val_loss: 0.1352
Epoch 4/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6130 - val_loss: 0.1376
Epoch 5/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6099 - val_loss: 0.1418
Epoch 6/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6065 - val_loss: 0.1466
Epoch 7/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6046 - val_loss: 0.1488
Epoch 8/15
145/145 [==============================] - 8s 52ms/step - loss: 1.5966 - val_loss: 0.1568
Epoch 9/15
145/145 [==============================] - 8s 52ms/step - loss: 1.5859 - val_loss: 0.1530
Epoch 10/15
145/145 [==============================] - 8s 52ms/step - loss: 1.6105 - val_l

INFO:tensorflow:Assets written to: Hybrid MP_wav/month4_10_step_lookahead_Hybrid\assets


INFO:tensorflow:Assets written to: Hybrid MP_wav/month4_10_step_lookahead_Hybrid\assets


63/63 [==============================] - 1s 19ms/step


D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [36]:
pd.concat(valdiation_metrics_dicts).mean()

RMSE     27.395974
MSE     935.968461
MAE      13.822675
MAPE      0.179238
R2        0.508363
dtype: float64